In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
import random

warnings.filterwarnings('ignore')

In [2]:
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

print(len(words))
print(max(len(word) for word in words))
words[:8]

32033
15


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# encoding letters to integers
charset = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(charset)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(f'{itos = }')
print(f'{vocab_size = }')

itos = {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
vocab_size = 27


In [4]:
import random
random.seed(42)
random.shuffle(words)

In [5]:
# Creating the dataset 
block_size = 8

def build_dataset(words):
    X = []
    Y = []
    for w in words:
        context = [0]*block_size 
        # print(w)
        for char in w + '.':
            X.append(context)
            Y.append(stoi[char])
            # print(f'{[itos[i] for i in context]} --> {char}')
            context = context[1:] + [stoi[char]]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y 

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
XDev, YDev = build_dataset(words[n1:n2])
Xts, Yts  = build_dataset(words[n2:])

torch.Size([182580, 8]) torch.Size([182580])
torch.Size([22767, 8]) torch.Size([22767])
torch.Size([22799, 8]) torch.Size([22799])


In [6]:
for x, y in zip(Xtr[:20], Ytr[:20]):
    print(''.join(itos[ix.item()] for ix in x),'--->', itos[y.item()])

........ ---> e
.......e ---> b
......eb ---> r
.....ebr ---> i
....ebri ---> m
...ebrim ---> a
..ebrima ---> .
........ ---> h
.......h ---> i
......hi ---> l
.....hil ---> t
....hilt ---> o
...hilto ---> n
..hilton ---> .
........ ---> j
.......j ---> h
......jh ---> e
.....jhe ---> n
....jhen ---> e
...jhene ---> .


In [7]:
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 
        self.bias = torch.randn(fan_out) if bias else None 

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias 

        return self.out 

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias]) 

In [8]:
class BatchNorm1d:
    def __init__(self, dim, eps = 1e-5, momentum = 0.1):
        self.eps = eps
        self.momentum = momentum 
        self.training = True
        # parameters trained with backprop
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

        # buffers updated with running momentum 
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim) 

    def __call__(self, x):
        if self.training:
            if x.ndim == 2:
                dim = 0 
            elif x.ndim == 3:
                dim = (0,1)
            xmean = x.mean(dim, keepdims=True)
            xvar = x.var(dim, keepdims=True)

        else:
            xmean = self.running_mean 
            xvar = self.running_var 

        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) 
        self.out = self.gamma * xhat + self.beta 

        if self.training:
            self.running_mean = (1. - self.momentum) * self.running_mean + self.momentum*xmean 
            self.running_var = (1. - self.momentum) * self.running_var + self.momentum*xvar

        return self.out 

    def parameters(self):
        return [self.gamma, self.beta] 

In [9]:
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

In [10]:
class Embedding:
    def __init__(self,vocab_size, embedding_dim):
        self.weight = torch.randn((vocab_size, embedding_dim))

    def __call__(self, x):
        self.out = self.weight[x]
        return self.out
        
    def parameters(self):
        return [self.weight]

In [11]:
class FlattenConsecutive:
    def __init__(self, n):
        self.n = n
        
    def __call__(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out

    def parameters(self):
        return []

In [12]:
class Sequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, x):
        self.out = x
        for layer in self.layers:
            self.out = layer(self.out)
        return self.out

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()] 

In [13]:
torch.manual_seed(42);

In [14]:
n_emb = 10
n_hidden = 68

model = Sequential([ 
    Embedding(vocab_size, n_emb),
    FlattenConsecutive(2), Linear(n_emb*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(n_hidden, vocab_size)   
])

with torch.no_grad():
    model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) 
for p in parameters:
    p.requires_grad=True

22397


In [ ]:
# optimization 
epochs = 200000
batch_size = 32
lossi = [] 
ud = []

for i in range(epochs):

    # minibatch construct 
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    xb, yb = Xtr[ix], Ytr[ix] 

    # Forward pass 
    logits = model(xb)
    loss = F.cross_entropy(logits, yb)

    for p in parameters:
        p.grad = None
    loss.backward() 

    # update 
    lr = 0.1 if i < 150000 else 0.01 
    for p in parameters:
        p.data += -lr*p.grad 

    # track stats 
    if i % 10000 == 0:
        print(f'{i:7d} / {epochs:7d}: {loss.item():.4f}')
    lossi.append(loss.item())

    with torch.no_grad():
        ud.append([(lr*p.grad.std() / p.data.std()).log10().item() for p in parameters]) # keeps a track of how large the updates are as compared to the actual values in parameters 

      0 /  200000: 3.8951
  10000 /  200000: 2.2082
  20000 /  200000: 2.4830
  30000 /  200000: 1.8707
  40000 /  200000: 2.1298
  50000 /  200000: 1.8637
  60000 /  200000: 1.8972
  70000 /  200000: 2.0881
  80000 /  200000: 1.5929
  90000 /  200000: 1.9128
 100000 /  200000: 2.1586
 110000 /  200000: 1.7666
 120000 /  200000: 1.9674
 130000 /  200000: 2.4003
 140000 /  200000: 1.9083
 150000 /  200000: 1.9191
 160000 /  200000: 2.1895
 170000 /  200000: 2.2663
 180000 /  200000: 2.0459


In [ ]:
plt.plot(torch.tensor(lossi).view(-1,1000).mean(1))

In [ ]:
for layer in model.layers:
    layer.training=False

In [ ]:
@torch.no_grad()
def split_loss(split):
    x, y = {'train': (Xtr, Ytr),
            'dev' : (XDev, YDev),
            'test' : (Xts, Yts)}[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
split_loss('train')
split_loss('dev')

In [ ]:
# inference from the model
g = torch.Generator().manual_seed(220 + 10)
for _ in range(10):
    out = []
    context = [0]*block_size
    while True:
        logits = model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))